In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
(train, val), info = tfds.load(
    "cats_vs_dogs",
    split=["train[:80%]", "train[80%:]"],
    with_info=True,
    as_supervised=True
)

In [3]:
def image_preprocessing(img, label):
    img = tf.image.resize(img, (resolution, resolution)) / 255.0
    return img, label

In [4]:
batch_size = 32
resolution = 224
num_examples = info.splits["train"].num_examples

train_batches = train.cache().shuffle(num_examples//4).map(image_preprocessing).batch(batch_size).prefetch(1)
val_batches = val.cache().map(image_preprocessing).batch(batch_size).prefetch(1)

In [5]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation="relu", input_shape=(resolution, resolution, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    
    tf.keras.layers.Conv2D(64, (3, 3), activation="relu"),
    tf.keras.layers.MaxPooling2D(2, 2),
    
    tf.keras.layers.Conv2D(128, (3, 3), activation="relu"),
    tf.keras.layers.MaxPooling2D(2, 2),
    
    tf.keras.layers.Conv2D(128, (3, 3), activation="relu"),
    tf.keras.layers.MaxPooling2D(2, 2),
    
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation="relu"),
    tf.keras.layers.Dense(2, activation="softmax")
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 222, 222, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 111, 111, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 109, 109, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 54, 54, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 52, 52, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 26, 26, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 24, 128)       1

In [6]:
model.compile(
    optimizer="adam",
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

In [7]:
number_of_epochs = 15

history = model.fit(
    train_batches,
    epochs=number_of_epochs,
    validation_data=val_batches
)

Epoch 1/15
582/582 [==============================] - 40s 54ms/step - loss: 0.6766 - accuracy: 0.5719 - val_loss: 0.6376 - val_accuracy: 0.6206
Epoch 2/15
582/582 [==============================] - 29s 49ms/step - loss: 0.5610 - accuracy: 0.7095 - val_loss: 0.4846 - val_accuracy: 0.7616
Epoch 3/15
582/582 [==============================] - 28s 48ms/step - loss: 0.4727 - accuracy: 0.7787 - val_loss: 0.3865 - val_accuracy: 0.8323
Epoch 4/15
582/582 [==============================] - 29s 50ms/step - loss: 0.3948 - accuracy: 0.8257 - val_loss: 0.3610 - val_accuracy: 0.8371
Epoch 5/15
582/582 [==============================] - 28s 49ms/step - loss: 0.3298 - accuracy: 0.8556 - val_loss: 0.3455 - val_accuracy: 0.8459
Epoch 6/15
582/582 [==============================] - 28s 49ms/step - loss: 0.2751 - accuracy: 0.8832 - val_loss: 0.3139 - val_accuracy: 0.8654
Epoch 7/15
582/582 [==============================] - 28s 49ms/step - loss: 0.2263 - accuracy: 0.9092 - val_loss: 0.2986 - val_accuracy:

In [8]:
url = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
feature_extractor = hub.KerasLayer(url, input_shape=(resolution, resolution, 3))

In [9]:
feature_extractor.trainable = False

In [11]:
model = tf.keras.models.Sequential([
    feature_extractor,
    tf.keras.layers.Dense(2, activation="softmax")
])

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 1280)              2257984   
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 2562      
Total params: 2,260,546
Trainable params: 2,562
Non-trainable params: 2,257,984
_________________________________________________________________


In [13]:
model.compile(
    optimizer="adam",
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

In [14]:
number_of_epochs = 3

model.fit(
    train_batches,
    epochs=number_of_epochs,
    validation_data=val_batches
)

Epoch 1/3
582/582 [==============================] - 168s 281ms/step - loss: 0.0973 - accuracy: 0.9611 - val_loss: 0.0569 - val_accuracy: 0.9809
Epoch 2/3
582/582 [==============================] - 95s 164ms/step - loss: 0.0312 - accuracy: 0.9899 - val_loss: 0.0366 - val_accuracy: 0.9877
Epoch 3/3
582/582 [==============================] - 50s 86ms/step - loss: 0.0247 - accuracy: 0.9915 - val_loss: 0.0399 - val_accuracy: 0.9865


In [15]:
export_path = "./models/cats_vs_dogs_transfer_learning.h5"
model.save(export_path)

In [16]:
load = tf.keras.models.load_model(
    export_path,
    custom_objects={"KerasLayer": hub.KerasLayer})

load.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 1280)              2257984   
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 2562      
Total params: 2,260,546
Trainable params: 2,562
Non-trainable params: 2,257,984
_________________________________________________________________


In [20]:
model_pred = model.predict(val_batches.take(1))

In [21]:
load_pred = load.predict(val_batches.take(1))

In [22]:
(abs(model_pred - load_pred)).max()

0.0